In [1]:
!pip install openai==0.27.8

In [42]:
import os
import openai

openai.api_key = 'api key 입력하세요'

In [43]:
import json

def get_current_weather(location, unit='celsius'): #unit='celsius' >> 셀시우스(온도 (섭씨))
    weather_info = {
        "location" : location,
        "temperature" : "24",
        "unit" : unit,
        "forecast" : ["sunny", "rainy"]

    }
    return json.dumps(weather_info)

In [44]:
get_current_weather('seoul')

'{"location": "seoul", "temperature": "24", "unit": "celsius", "forecast": ["sunny", "rainy"]}'

In [45]:
def get_gps():
    return '서울'

In [46]:
#함수 호출 dict()

available_functions = {
    'get_current_weather' : get_current_weather,
    'get_location_from_gps' : get_gps
}

In [47]:
available_functions['get_current_weather']('서울')

'{"location": "\\uc11c\\uc6b8", "temperature": "24", "unit": "celsius", "forecast": ["sunny", "rainy"]}'

In [48]:
available_functions['get_current_weather']('seoul')


'{"location": "seoul", "temperature": "24", "unit": "celsius", "forecast": ["sunny", "rainy"]}'

In [49]:
available_functions['get_location_from_gps']()

'서울'

In [52]:
def run_conversation(input_texts):
    # 1. convesation + available_funtions (함수를 호출하는 기능) >> gpt
    messages = [
        {'role' : 'user', 'content' : input_texts}
    ]

    functions = [
        {
            "name" : "get_current_weather",
            "description" : "Get the current weather in a given location",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "location" : {
                        "type" : "string",
                        "description" : "The city and state, e.g. San Francisco, CA"
                    },
                    "unit":{
                        "type" : "string",
                        "enum" : ["celsius", "fahrenheit"] #섭씨/화씨
                    },
                    },
                "required" : ['location'],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = messages,
        functions = functions,
        function_call = 'auto'
    )
    # print(response)
    response_message = response['choices'][0]['message']
    # print(response_message)

    #function_call : 함수 특징
    if response_message.get('function_call'):

        #함수 이름과 인자 가져옴
        func_name = response_message['function_call']['name']
        func_args = json.loads(response_message['function_call']['arguments'])

        #response_message : function_call, name, arguments
        messages.append(response_message)

        #함수 호출 >> 결과값
        func_response = available_functions[func_name](
            location = func_args.get('location'),
            unit = func_args.get('unit'))

        #함수 호출 결과 >> 메시지에 추가
        messages.append({
            "role" : "function",
            "name" : func_name,
            "content" : func_response
        })

        #역할 : 함수 호출결과는 역할을 function으로 설정
        # print(messages)

        second_response = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo',
            messages = messages,
       )
        # 사용자 요청사항, 함수 이름 및 인자, 함수 호출 결과
        second_response = second_response['choices'][0]['message']
        return second_response
    else:
        return response

In [37]:
run_conversation("오늘 날씨는?")
#response

{
  "id": "chatcmpl-9lzWUa7MNazhLLLtvUpb0ezH80zKt",
  "object": "chat.completion",
  "created": 1721224938,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc5b4\ub290 \uc9c0\uc5ed\uc758 \ub0a0\uc528\ub97c \uc54c\uace0 \uc2f6\uc73c\uc2e0\uac00\uc694?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 84,
    "completion_tokens": 24,
    "total_tokens": 108
  },
  "system_fingerprint": null
}


In [35]:
run_conversation("오늘 날씨는?")
# response_message

{
  "role": "assistant",
  "content": "\uc5b4\ub5a4 \ub3c4\uc2dc\uc758 \ub0a0\uc528 \uc815\ubcf4\ub97c \ud655\uc778\ud558\uace0 \uc2f6\uc73c\uc2e0\uac00\uc694?"
}


In [51]:
run_conversation("How is the weather in Seoul?")
# print(messages)

[{'role': 'user', 'content': 'How is the weather in Seoul?'}, <OpenAIObject at 0x7c6cb097bce0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Seoul\",\"unit\":\"celsius\"}"
  }
}, {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "Seoul", "temperature": "24", "unit": "celsius", "forecast": ["sunny", "rainy"]}'}]


In [54]:
run_conversation("How is the weather in Seoul?")

<OpenAIObject at 0x7c6cb09192b0> JSON: {
  "role": "assistant",
  "content": "The weather in Seoul is currently 24\u00b0C with a mix of sun and rain."
}

Refactoring(기존코드수정)

In [56]:
def run_conversation_2(input_texts):
    # 1. convesation + available_funtions (함수를 호출하는 기능) >> gpt
    messages = [
        {'role' : 'user', 'content' : input_texts}
    ]

    functions = [
        {
            "name" : "get_current_weather",
            "description" : "Get the current weather in a given location",
            "parameters" : {
                "type" : "object",
                "properties" : {
                    "location" : {
                        "type" : "string",
                        "description" : "The city and state, e.g. San Francisco, CA"
                    },
                    "unit":{
                        "type" : "string",
                        "enum" : ["celsius", "fahrenheit"] #섭씨/화씨
                    },
                    },
                "required" : ['location'],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = messages,
        functions = functions,
        function_call = 'auto'
    )
    # print(response)
    response_message = response['choices'][0]['message']
    # print(response_message)

    #function_call : 함수 특징
    if response_message.get('function_call'):

        #함수 이름과 인자 가져옴
        func_name = response_message['function_call']['name']
        func_args = json.loads(response_message['function_call']['arguments'])

        #함수 호출
        # >> 어떤 함수를 호촐되는지 모르기 때문에, 즉 인자를 특정할 수 없다.
        func_response = available_functions[func_name](**func_args)

        messages.append(response_message)

        messages.append({
            "role" : "function",
            "name" : func_name,
            "content" : func_response
        })

        second_response = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo',
            messages = messages,
       )
        # 사용자 요청사항, 함수 이름 및 인자, 함수 호출 결과
        second_response = second_response['choices'][0]['message']
        return second_response
    else:
        return response

In [57]:
run_conversation_2("How is the weather in Seoul?")

<OpenAIObject at 0x7c6cb08dc2c0> JSON: {
  "role": "assistant",
  "content": "The current weather in Seoul is 24 degrees Celsius with a mix of sunny and rainy conditions."
}

In [62]:
run_conversation('현재 위치는 어디인가요?')

<OpenAIObject chat.completion id=chatcmpl-9lzpYVTg37fgiJBPxcBYgQxkZ6Muz at 0x7c6cb07c0bd0> JSON: {
  "id": "chatcmpl-9lzpYVTg37fgiJBPxcBYgQxkZ6Muz",
  "object": "chat.completion",
  "created": 1721226120,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\uc8c4\uc1a1\ud569\ub2c8\ub2e4. \ud604\uc7ac \uc704\uce58\ub97c \uc54c \uc218 \uc5c6\uc2b5\ub2c8\ub2e4. \uc0ac\uc6a9\uc790 \uc704\uce58\ub97c \uc81c\uacf5\ud574 \uc8fc\uc2dc\uac70\ub098 \uc6d0\ud558\ub294 \uc704\uce58\ub97c \ub9d0\uc500\ud574 \uc8fc\uc2dc\uba74 \ud574\ub2f9 \uc704\uce58\uc758 \ud604\uc7ac \ub0a0\uc528\ub97c \uc54c\ub824\ub4dc\ub9b4 \uc218 \uc788\uc2b5\ub2c8\ub2e4."
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 85,
    "completion_tokens": 65,
    "total_tokens": 150
  },
  "system_fingerprint": null
}

In [64]:
message = run_conversation_2("현재 위치는 어디인가요?")
message.choices[0].message.content

'죄송합니다. 현재 위치를 알 수 없습니다. 위치를 입력해 주세요.'

In [66]:
message = run_conversation_2("서울의 날씨는 어떤가요?")
message.content


'서울의 현재 온도는 24도입니다. 날씨는 대체로 맑지만 가끔 소나기가 올 수 있습니다.외출 시 우산을 챙기시는 것이 좋을 것 같아요.'